In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import warnings
warnings.simplefilter('ignore')

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import neighbors

from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
INPUT_PATH = "/content/drive/MyDrive/GCI/titanic/Input"

train = pd.read_csv(os.path.join(INPUT_PATH, "train.csv"))
test = pd.read_csv(os.path.join(INPUT_PATH, "test.csv"))
train["Perished"] = train["Perished"].replace({1:0, 0:1})

data = pd.concat([train, test], ignore_index = True)

# 特徴量作成

In [3]:
data["Title"] =  data["Name"].apply(lambda x:x.split(',')[1].split('.')[0].strip())

mapping = {
    'Mlle': 'Miss', 
    'Major': 'Mr', 
    'Col': 'Mr', 
    'Sir': 'Mr', 
    'Don': 'Mr', 
    'Mme': 'Miss',
    'Jonkheer': 'Mr', 
    'Lady': 'Mrs', 
    'Capt': 'Mr', 
    'Countess': 'Mrs', 
    'Ms': 'Miss', 
    'Dona': 'Mrs'
    }

data.replace({'Title': mapping}, inplace=True)

titles = ['Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Rev']
for title in titles:
    age_to_impute = data.groupby('Title')['Age'].median()[titles.index(title)]
    data.loc[(data['Age'].isnull()) & (data['Title'] == title), 'Age'] = age_to_impute

In [4]:
data['Last_Name'] = data['Name'].apply(lambda x: str.split(x, ",")[0])
data['Family_Survival'] = 0.5

for grp, grp_df in data[['Perished','Name', 'Last_Name', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Last_Name', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Perished'].max()
            smin = grp_df.drop(ind)['Perished'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 0

print("Number of passengers with family survival information:", 
      data.loc[data['Family_Survival']!=0.5].shape[0])

Number of passengers with family survival information: 420


In [5]:
for _, grp_df in data.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Perished'].max()
                smin = grp_df.drop(ind)['Perished'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(data[data['Family_Survival']!=0.5].shape[0]))

Number of passenger with family/group survival information: 546


In [6]:
data['Fare'].fillna(data['Fare'].median(), inplace = True)
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1
data['Sex'].replace(['male','female'],[0,1],inplace=True)

# Making Bins
# Fare
data['FareBin'] = pd.qcut(data['Fare'], 5)
label = LabelEncoder()
data['FareBin_Code'] = label.fit_transform(data['FareBin'])
# Age
data['AgeBin'] = pd.qcut(data['Age'], 4)
label = LabelEncoder()
data['AgeBin_Code'] = label.fit_transform(data['AgeBin'])

# KNN

In [7]:
train = data[data["Perished"].notnull()][[
    "Perished","Pclass","Sex","Family_Size",
    "Family_Survival","FareBin_Code","AgeBin_Code"
    ]]
test = data[data["Perished"].isnull()][[
    "Pclass","Sex","Family_Size","Family_Survival",
    "FareBin_Code","AgeBin_Code"
    ]]

In [8]:
X = train.drop('Perished', 1)
y = train['Perished']
X_test = test.copy()

std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)
X_test = std_scaler.transform(X_test)

In [9]:
# Best params
knn = KNeighborsClassifier(algorithm='auto', leaf_size=26, metric='minkowski', 
                           metric_params=None, n_jobs=1, n_neighbors=6, p=2, 
                           weights='uniform')
knn.fit(X, y)
y_pred = knn.predict(X_test)

# 提出

In [10]:
# for ensemble
submissions = pd.read_csv("/content/drive/MyDrive/GCI/titanic/Input/gender_submission.csv")
submissions["Perished"] = y_pred.astype('int64')
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/ensemble/KNN-0.8181.csv", index=False)

# for kaggle
submissions = submissions.rename(columns={"Perished":"Survived"})
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/kaggle/KNN-0.8181.csv", index=False)
# Kaggle -> 0.8181

# for GCI
submissions = submissions.rename(columns={"Survived":"Perished"})
submissions["Perished"] = submissions["Perished"].replace({1:0, 0:1})
submissions.to_csv("/content/drive/MyDrive/GCI/titanic/Output/gci/GCI-KNN-0.8181csv", index=False)

# 参考・引用
*  https://www.kaggle.com/code/konstantinmasich/titanic-0-82-0-83/notebook